# 03 Data Cleaning

## Notebook Overview

This notebook consolidates, standardizes, and resamples the raw dataset to create a clean hourly-level time series for modeling.

**Key Steps:**

* **Column cleanup:** Drops duplicates (`House overall`, `Solar`), merges features (e.g. `Furnace 1+2` → `furnace`, kitchen zones → `kitchen`)
* **Resampling:** Aggregates numeric features via hourly mean and categorical values via mode
* **Datetime features:** Adds granular temporal features (hour, weekday, time-of-day label) to support time-aware modeling
* **Noise handling:** Fixes format inconsistencies (`cloudCover`), and removes multicollinear features (`dewPoint`, `apparentTemperature`, etc.)
* **Standardized naming:** Renames all features to snake\_case for consistency

> Output: `data_cleaned.csv` — a tidy, hourly dataset with interpretable, model-ready features and minimal redundancy.

### Thoughts, Tradeoffs & Considerations

* **Resampling decision:** Original data is logged per second — too granular for typical energy forecasting tasks. High-frequency noise drowns out meaningful consumption patterns. Hourly resampling strikes a pragmatic balance between signal richness and model stability.
* **Feature consolidation:** Merging `Furnace 1/2` and the three `Kitchen` zones was necessary, these sub-meters represent the same functional unit. Keeping them separate just inflated dimensionality without adding modeling value.
* **Weather multicollinearity:** Features like `apparentTemperature` and `dewPoint` were dropped after correlation checks, they’re heavily dependent on `temperature` and `humidity`. Including all would confuse models or force regularization.
* **`cloudCover` is a weird one:** Though it's numeric, values are clearly bucketed (0.0, \~0.3, 1.0). It’s likely categorical in nature, but we left it as float for now. Might revisit with binning or one-hot if it proves useful.
* **Datetime augmentation:** Added classic features like `hour`, `weekday`, and a `timing` label (e.g. Morning/Evening) to enable pattern-aware modeling. These are cheap to compute and often boost tree-based models.
* **Mode aggregation quirks:** Non-numeric columns (like `summary`) were resampled using mode, but this approach can fail if values are missing or evenly split. For now, only kept them if they behaved well, the rest were dropped.
* **Naming standardization:** Renamed all columns to snake\_case, stripped units. This avoids parsing issues in pipelines and aligns with Pythonic conventions. Small thing, but big downstream win.
* **Data loss tradeoff:** Dropping rows with missing timestamps or all-NaN values post-resampling loses some data, but keeping broken or empty rows would introduce bigger downstream problems.

> The main challenge here was **retaining signal while simplifying**. The raw dataset is rich but noisy — this cleaning pass ensures what we keep is interpretable, numerically sound, and ready for modeling.

In [1]:
import pandas as pd
import numpy as np
from typing import List

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the column width and overall display width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)

In [4]:
# df: pd.DataFrame = pd.read_csv('../data/raw/Data Smart Meter.csv')
df: pd.DataFrame = pd.read_parquet('../data/raw/data_smart_meter.parquet')
df.head()

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],Kitchen 14 [kW],Kitchen 38 [kW],Barn [kW],Well [kW],Microwave [kW],Living room [kW],Solar [kW],temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0


In [5]:
df.columns = [i.replace(' [kW]', '') for i in df.columns]

In [6]:
df = df.dropna(subset=['time'])

# Convert valid timestamps to datetime
# Treat each row as a minute step from fixed start
start_time = pd.Timestamp("2016-01-01 05:00:00")
df = df.reset_index(drop=True)  # drop old index if necessary
df["time"] = pd.date_range(start=start_time, periods=len(df), freq="T")
df = df.set_index('time')

In [7]:
df['Furnace'] = df[['Furnace 1','Furnace 2']].sum(axis=1)
df['Kitchen'] = df[['Kitchen 12','Kitchen 14','Kitchen 38']].mean(axis=1)
df.drop(['Furnace 1','Furnace 2','Kitchen 12','Kitchen 14','Kitchen 38','icon','summary'], axis=1, inplace=True)

In [8]:
df['cloudCover'].replace(['cloudCover'], method='bfill', inplace=True)
df['cloudCover'] = df['cloudCover'].astype('float')
df.head(3)

,use,gen,House overall,Dishwasher,Home office,Fridge,Wine cellar,Garage door,Barn,Well,Microwave,Living room,Solar,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability,Furnace,Kitchen
time,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 05:00:00,0.932833,0.003483,0.932833,0.000033,0.442633,0.124150,0.006983,0.013083,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.082617,0.000189
2016-01-01 05:01:00,0.934333,0.003467,0.934333,0.000000,0.444067,0.124000,0.006983,0.013117,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.084533,0.000189
2016-01-01 05:02:00,0.931817,0.003467,0.931817,0.000017,0.446067,0.123533,0.006983,0.013083,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.083017,0.000206


In [9]:
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.day_name()
df['weekofyear'] = df.index.isocalendar().week
df['hour'] = df.index.hour
df['minute'] = df.index.minute
df.head(3)

,use,gen,House overall,Dishwasher,Home office,Fridge,Wine cellar,Garage door,Barn,Well,Microwave,Living room,Solar,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability,Furnace,Kitchen,year,month,day,weekday,weekofyear,hour,minute
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 05:00:00,0.932833,0.003483,0.932833,0.000033,0.442633,0.124150,0.006983,0.013083,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.082617,0.000189,2016,1,1,Friday,53,5,0
2016-01-01 05:01:00,0.934333,0.003467,0.934333,0.000000,0.444067,0.124000,0.006983,0.013117,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.084533,0.000189,2016,1,1,Friday,53,5,1
2016-01-01 05:02:00,0.931817,0.003467,0.931817,0.000017,0.446067,0.123533,0.006983,0.013083,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.083017,0.000206,2016,1,1,Friday,53,5,2


In [10]:
bins = [-1, 3, 11, 16, 21, 24]
labels = ["Night", "Morning", "Afternoon", "Evening", "Night"]

In [11]:
def map_hour_to_timing(hour_series: pd.Series) -> pd.Series:
    """Categorize hour-of-day (0–23) into named time segments."""
    hour_series = hour_series.astype(int)
    conditions = [
        hour_series.isin([22, 23, 0, 1, 2, 3]),
        hour_series.between(4, 11),
        hour_series.between(12, 16),
        hour_series.between(17, 21)
    ]
    choices = ["Night", "Morning", "Afternoon", "Evening"]
    return pd.Series(np.select(conditions, choices, default="Unknown"), index=hour_series.index)

In [12]:
# Resample only numeric columns using 15-minute mean
numeric_cols = df.select_dtypes(include='number').columns
df_resampled = df[numeric_cols].resample('h').mean()

#  keep categorical columns, take the most frequent value (mode)
categorical_cols: List = df.select_dtypes(include='object').columns
if len(categorical_cols) > 0:
    df_mode = df[categorical_cols].resample('h').agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    df_resampled = df_resampled.join(df_mode)

df_resampled = df_resampled.dropna(how='all')

In [13]:
# Convert discretized cloudCover values to numeric
df_resampled['cloudCover'] = pd.to_numeric(df_resampled['cloudCover'], errors='coerce')

In [14]:
# drop columns with multicollinearity: `precipIntensity`, `Solar [kW]`, `House overall [kW], `apparentTemperature`, and `dewPoint` since the provided information is already contained in other features
df_resampled = df_resampled.drop(columns={"precipIntensity", "Solar", "House overall","apparentTemperature", "dewPoint"})

In [15]:
df_resampled = df_resampled.rename(columns={
    "use": "use_house_overall",
    "gen": "generated_solar",
    "Dishwasher": "dishwasher",
    "Home office": "homeoffice",
    "Fridge": "fridge",
    "Wine cellar": "winecellar",
    "Garage door": "garagedoor",
    "Barn": "barn",
    "Well": "well",
    "Microwave": "microwave",
    "Living room": "livingroom",
    "Furnace": "furnace",
    "Kitchen": "kitchen",
    "cloudCover": "cloudcover",
    "windBearing": "windbearing",
    "windSpeed": "windspeed",
    "precipProbability": "precipprobability"
})

In [16]:
df_resampled["hour"] = df_resampled.index.hour
df_resampled["timing"] = map_hour_to_timing(df_resampled["hour"])

In [17]:
df_resampled.head(3)

,use_house_overall,generated_solar,dishwasher,homeoffice,fridge,winecellar,garagedoor,barn,well,microwave,livingroom,temperature,humidity,visibility,pressure,windspeed,cloudcover,windbearing,precipprobability,furnace,kitchen,year,month,day,weekofyear,hour,minute,weekday,timing
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 05:00:00,1.044130,0.003307,0.000064,0.241814,0.037861,0.063351,0.013046,0.038881,0.001042,0.021652,0.001505,36.131000,0.619667,10.0,1016.888,9.150333,0.75,282.100000,0.0,0.393188,0.000274,2016.0,1.0,1.0,53.0,5,29.5,Friday,Morning
2016-01-01 06:00:00,0.918167,0.003422,0.000099,0.043294,0.075522,0.112942,0.012836,0.039181,0.001021,0.004216,0.001618,35.838667,0.610000,10.0,1016.232,8.284000,0.75,284.733333,0.0,0.456708,0.000250,2016.0,1.0,1.0,53.0,6,29.5,Friday,Morning
2016-01-01 07:00:00,0.714736,0.003448,0.000043,0.043416,0.059486,0.007184,0.013299,0.034439,0.001014,0.004246,0.001629,35.385000,0.613000,10.0,1015.989,7.927000,0.75,279.400000,0.0,0.372170,0.000242,2016.0,1.0,1.0,53.0,7,29.5,Friday,Morning


In [18]:
df_resampled.to_csv("../data/interim/data_cleaned.csv")